## Introduction

Washington State University's Decision Aid System as well as others have provided tree fruit growers with population phenology models for the common pest known as codling moth (_Cydia pomonella_) for years which have been shown to be quite reliable, and much research has been done into the population dynamics of codling moth subject to the manipulation various independent variables. One inconsistency relayed to the WSU CAHNRS Dept. of Entomology by growers is a discrepancy in the time our models expect the first emergences of the overwintering generation, and when their traps actually find evidence of them. Some organizations have asserted that these first trap captures are indicitive of the _real_ first emergence times, stating that they should be used as a sort of "biofix" that the models must be translated over to match. The fact that the same growers report significantly better matching in the second and third generations suggests to us that this is not the case, and we hypothesize that it is the result of sampling error that causes the difference in time between first emergence and first capture in these systems. Due to the very monolithic management of real orchards making field data not useful, and the very long turnaround times for new lab data to become available, it has been opted to use a simulation generating data matching key expectations gathered in previous lab studies to investigate the problem. The simulation is used to generate a swathe of data as a convolution of three varying degrees of freedom in the starting conditions (starting population size of the overwintering generation, percentage chance to capture a moth on a given degree day, and the average mating delay experienced by the moths).

## Import libraries and split into individual dataframes for each permutation of variables

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
df = pd.read_csv('test_1.csv')
dfs = np.array_split(df, len(df)/df.dd_span.head(1).iloc[0])

## Compile data about capture-emergence difference and clean instances with no first generation captures and/or first generation emergences

In [ ]:
def compile_diffs(dfs, thresh: int):
    capture_data = []
    for frame in dfs:
        first_capture = frame[frame.captured > thresh].head(1).dd
        first_emergence = frame[(frame.pop_active_0 > thresh)].head(1).dd
        if not first_capture.empty and not first_emergence.empty:
            diff = first_capture.iloc[0] - first_emergence.iloc[0]
            pop_0 = frame.head(1).pop_0.iloc[0]
            prob_capture = frame.head(1).prob_capture.iloc[0]
            mating_delay = frame.head(1).mating_delay.iloc[0]
            ratio_naive = pop_0 * prob_capture
            ratio = (pop_0 ** 1/3) * (prob_capture ** 3)
            capture_data.append((diff, ratio, ratio_naive, pop_0, prob_capture, mating_delay))
    return capture_data

first_capture_data = compile_diffs(dfs, 1)

Two ratios are generated here for combinations between overwintering pop size and capture probability, a naive ratio just by multiplying them together (yes, I know this technically isn't a ratio since it's not divison, but since one of these values is very big and the other very small, multiplying gives much nicer numbers and is no less valid as a combination of the two than if I chose to multiply), and a better ratio by multiplying the cube root of the population to the cube of the capture probability. This balances out their relative importance in determining the difference much better, as will be demonstrated.

## Organize data into three dataframes, a unified frame, an explanitory varaible frame, and a dependent variable frame

In [ ]:
capture_df = pd.DataFrame(first_capture_data, columns=['difference', 'ratio', 'naive_ratio', 'pop_0', 'prob_capture', 'mating_delay'])
df_x = capture_df.drop('difference', axis=1)
df_y = capture_df['difference']

## View variables against each other

In [ ]:
capture_df.plot.scatter(1, 0, 1.5, logy=False)
capture_df.plot.scatter(2, 0, 1.5, logy=False)
capture_df.plot.scatter(3, 0, 1.5, logy=False)
capture_df.plot.scatter(4, 0, 1.5, logy=False)
capture_df.plot.scatter(5, 0, 1.5, logy=False)

## Create function for automatically fitting reciprocal functions using inverse-composition linearization

In [ ]:
def autofit_reciprocal(x, y, a_guess, b_guess, c_guess, full_output=False):
    a = a_guess
    b = b_guess
    c = c_guess
    slope = 2
    intercept = 1

    # fit 'b' parameter
    magnitude = 1
    for _ in range(1,50):
        y_linear = np.power(np.multiply(np.reciprocal(np.subtract(y, b)), a), 1 / c)
        p = np.polyfit(x, y_linear, 1, w=np.log(y))
        b_below = b + (0.1 ** magnitude)
        b_above = b - (0.1 ** magnitude)
        y_linear_above = np.power(np.multiply(np.reciprocal(np.subtract(y, b_above)), a), 1 / c)
        y_linear_below = np.power(np.multiply(np.reciprocal(np.subtract(y, b_below)), a), 1 / c)
        p_above = np.polyfit(x, y_linear_above, 1, w=np.log(y))
        p_below = np.polyfit(x, y_linear_below, 1, w=np.log(y))
        if abs(p[1]) < abs(p_above[1]) and abs(p[1]) < abs(p_below[1]):
            magnitude = magnitude + 1
        elif abs(p_above[1]) < abs(p_below[1]):
            b = b_above
        elif abs(p_below[1]) < abs(p_above[1]):
            b = b_below

    # fit 'a' parameter
    magnitude = 0
    for _ in range(1,50):
        y_linear = np.power(np.multiply(np.reciprocal(np.subtract(y, b)), a), 1 / c)
        p = np.polyfit(x, y_linear, 1, w=np.log(y))
        a_below = a + (0.1 ** magnitude)
        a_above = a - (0.1 ** magnitude)
        y_linear_above = np.power(np.multiply(np.reciprocal(np.subtract(y, b)), a_above), 1 / c)
        y_linear_below = np.power(np.multiply(np.reciprocal(np.subtract(y, b)), a_below), 1 / c)
        p_above = np.polyfit(x, y_linear_above, 1, w=np.log(y))
        p_below = np.polyfit(x, y_linear_below, 1, w=np.log(y))
        # print(f'{a}, {p_above[0]}, {p[0]}, {p_below[0]}')
        if abs(p[0] - 1) < abs(p_above[0] - 1) and abs(p[0] - 1) < abs(p_below[0] - 1):
            magnitude = magnitude + 1
        elif abs(p_above[0] - 1) < abs(p_below[0] - 1):
            a = a_above
        elif abs(p_below[0] - 1) < abs(p_above[0] - 1):
            a = a_below
    if full_output:
        return (a, b, p)
    else:
        return (a, b)

## Fit curve to data using naive ratio

In [ ]:
x = capture_df.naive_ratio
y = capture_df.difference

c = 0.29
a, b, p = autofit_reciprocal(x, y, 3, 0.3, c, full_output=True)
print(*p)

y_linear_naive = np.power(np.multiply(np.reciprocal(np.subtract(y, b)), a), 1 / c)

fitted_x = np.linspace(x.min(), x.max(),1000)
fitted_y_naive = np.add(np.multiply(np.reciprocal(np.power(x, c)), a), b)

ax = plt.axes()
ax.scatter(x, y, label='Data', s=2)
ax.scatter(x, fitted_y_naive, label='Prediction', s=1)
ax.set_title('Fitted curve for ratio to difference relationship')
ax.set_ylabel('ln( difference )')
ax.set_yscale('log') # view in log to emphasize small differences
ax.set_ylim(1, 80)
ax.set_xlabel('ratio')
ax.legend()

## Check residuals

In [ ]:
ax = plt.axes()
ax.scatter(x, np.subtract(y, fitted_y_naive), label='Data', s=2)
ax.set_title('Residuals vs. ratio')
ax.set_ylabel('Actual value - predicted value')
ax.set_xlabel('ratio')

+/- 10 degree day heteroscedasticity could definitely be improved

## Fit curve to data using smarter ratio

In [ ]:
x = capture_df.ratio
y = capture_df.difference

c = 0.2
a, b = autofit_reciprocal(x, y, 3, 0.7, c)

y_linear = np.power(np.multiply(np.reciprocal(np.subtract(y, b)), a), 1 / c)

fitted_x = np.linspace(x.min(), x.max(),1000)
fitted_y = np.add(np.multiply(np.reciprocal(np.power(x, c)), a), b)

ax = plt.axes()
ax.scatter(x, y, label='Data', s=2)
ax.scatter(x, fitted_y, label='Prediction', s=1)
ax.set_title('Fitted curve for ratio to difference relationship')
ax.set_ylabel('ln( difference )')
ax.set_yscale('log') # view in log to emphasize small differences
ax.set_ylim(1, 80)
ax.set_xlabel('ratio')
ax.legend()
print((a, b))

For the ratio function of (pop_0^1/3) * (prob_capture^3), a function of y = (3.024 / x ^ 0.2) + 0.965 predicts the difference in first emergence and first capture.

## Determine goodness of fit by looking at residuals

### Residual Plot

In [ ]:
ax = plt.axes()
ax.scatter(x, np.subtract(y, fitted_y), label='Data', s=2)
ax.set_title('Residuals vs. ratio')
ax.set_ylabel('Actual value - predicted value')
ax.set_xlabel('ratio')

Definitely still some heteroscedasticity, but it looks like the worst of it is only +/- 3, and the majority of points seem to be in the +/- 1 area. Measureable improvement from the naive ratio. Furthermore, from looking at the fitted curve graphs of the two, no value of 'c' was able to match both the extreme and median points to a satisfactory degree with the naive ratio, but a 'c' of 0.2 matches both quite well with the smarter combination

### Measure how bad the heteroscedasticity is within the unstable small values area

In [ ]:
from scipy.ndimage import gaussian_filter1d

rolling_avg = gaussian_filter1d(np.subtract(y, fitted_y), 15, mode='nearest')

ax = plt.axes()
ax.scatter(x, np.subtract(y, fitted_y), label='Data', s=2)
ax.scatter(x, rolling_avg, label='Gaussian Rolling Avg', s=1)
ax.set_title('Residuals vs. ratio')
ax.set_ylabel('Actual value - predicted value')
ax.set_xlabel('ratio')
ax.legend()

With a gaussian weighted rolling average with a standard deviation of 15, it does appear the vast majority of all residuals hang within the +/- 1.0 range, which is definitely below noise for field emergences/captures. I am quite satisfied with this, though I do think the ultimate root cause of the heteroscedasticity could probably be weeded out given enough time and the residuals could be made 100% normally distributed. For now, this is well within the bounds of utility.

### Average bias per point

In [ ]:
np.sum(np.subtract(y, fitted_y)) / len(x)

The fitted curve is a little biased negative relative to the actual data, but I again feel that less than a fifth of a degree day average negative bias is satisfactory for practical use.

## Investigate how the difference evolves when looking at second captures, third captures, etc.

### Compile 1st through 30th differences

In [ ]:
diffs = []
for i in range(1,31):
    row = compile_diffs(dfs, i)
    row_df = pd.DataFrame(row, columns=['difference', 'ratio', 'naive_ratio', 'pop_0', 'prob_capture', 'mating_delay'])
    diffs.append(row_df)

len(diffs)

### Look at the graphs of the second, third, tenth, and thirtieth differences against the ratio

In [ ]:
diffs[1].plot.scatter(1, 0, 1.5, title='Second diffs', logy=True)
diffs[2].plot.scatter(1, 0, 1.5, title='Third diffs', logy=True)
diffs[4].plot.scatter(1, 0, 1.5, title='FIfth diffs', logy=True)
diffs[9].plot.scatter(1, 0, 1.5, title='Tenth diffs', logy=True)
diffs[19].plot.scatter(1, 0, 1.5, title='Twentieth diffs', logy=True)
diffs[29].plot.scatter(1, 0, 1.5, title='Thirtieth diffs', logy=True)

It looks like the variance for small ratios gets greater as the difference being inspected increases